## 4.4 집계
### 4.4.1. 집계 기본
* 검색 API 요청 본문에 `agg` 추가해 집계 수행
* `size` 0 지정 시
  * 검색 상위에 매칭된 문서를 받을 수 없음
  * 조건에 매치되는 모든 문서를 집계에 사용할 수 있음
  * 성능에 유리
    * 상위 문서 내용을 모을 필요 없음
    * 점수 계산하는 과정 생략
    * 캐시 도움을 더 많이 받을 수 있음     
* 한 번의 요청에 여러 집계를 요청할 수 있기 때문에 이름을 붙여 구분
* 검색 쿼리에 매칭된 모든 문서에 대해 수행되어 클러스터 성능을 급격히 저하시킬 수 있음
  * 키바나 사용 시 과도한 집계를 사용하는 무거운 그래프를 생성하거나 다수의 사용자가 동시 접속해 문제를 야기할 수 있음  

In [3]:
%%bash
curl -X GET --location "http://localhost:9200/kibana_sample_data_ecommerce/_search" \
    -H "Content-Type: application/json" \
    -d "{
          \"size\": 0,
          \"query\": {
            \"term\": {
              \"currency\": {
                \"value\": \"EUR\"
              }
            }
          },
          \"aggs\": {
            \"my-sum-aggregation-name\": {
              \"sum\": {
                \"field\": \"taxless_total_price\"
              }
            }
          }
        }"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   583    0   233  100   350  14770  22187 --:--:-- --:--:-- --:--:-- 38866


{"took":11,"timed_out":false,"_shards":{"total":1,"successful":1,"skipped":0,"failed":0},"hits":{"total":{"value":4675,"relation":"eq"},"max_score":null,"hits":[]},"aggregations":{"my-sum-aggregation-name":{"value":350884.12890625}}}

### 4.4.2. 메트릭 집계
* 문서에 대한 산술적인 연산 수행

#### avg, max, min, sum 집계
* 매칭된 문서를 대상으로 지정된 필드 값 가져온 후 각각 평균, 최댓값, 최솟값, 합 계산해 반환

#### stats 집계
* 다중 값 숫자 메트릭 집계 (multi-value numeric metric aggregation)
  * 여러 숫자값을 한 번에 반환하는 메트릭 집계  
* 지정된 필의 평균, 최댓값, 최솟값, 합, 개수를 모두 계산해 반환

In [4]:
%%bash
curl -X GET --location "http://localhost:9200/kibana_sample_data_ecommerce/_search" \
    -H "Content-Type: application/json" \
    -d "{
          \"size\": 0,
          \"query\": {
            \"term\": {
              \"currency\": {
                \"value\": \"EUR\"
              }
            }
          },
          \"aggs\": {
            \"my-sum-aggregation-name\": {
              \"stats\": {
                \"field\": \"taxless_total_price\"
              }
            }
          }
        }"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   650    0   298  100   352  16829  19879 --:--:-- --:--:-- --:--:-- 38235


{"took":12,"timed_out":false,"_shards":{"total":1,"successful":1,"skipped":0,"failed":0},"hits":{"total":{"value":4675,"relation":"eq"},"max_score":null,"hits":[]},"aggregations":{"my-sum-aggregation-name":{"count":4675,"min":6.98828125,"max":2250.0,"avg":75.05542864304813,"sum":350884.12890625}}}

#### cardinality 집계
* 지정된 필드가 가진 고유한 값의 개수를 [HyperLogLog++](https://d2.naver.com/helloworld/711301) 알고리즘을 사용해 추정한 근사값
* 정확도 조절을 위해서는 `precision_threshold` 사용
  * 해당 값을 높이면 정확도가 올라가지만, 메모리 사용이 높아짐
  * 최종 cardinality보다 큰 값으로 설정  

In [3]:
%%bash
curl -X GET --location "http://localhost:9200/kibana_sample_data_ecommerce/_search" \
    -H "Content-Type: application/json" \
    -d "{
          \"size\": 0,
          \"query\": {
            \"term\": {
              \"currency\": {
                \"value\": \"EUR\"
              }
            }
          },
          \"aggs\": {
            \"my-cardinality-aggregation-name\": {
              \"cardinality\": {
                \"field\": \"customer_id\",
                \"precision_threshold\": 3000
              }
            }
          }
        }"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   631    0   228  100   403  11861  20965 --:--:-- --:--:-- --:--:-- 33210


{"took":14,"timed_out":false,"_shards":{"total":1,"successful":1,"skipped":0,"failed":0},"hits":{"total":{"value":4675,"relation":"eq"},"max_score":null,"hits":[]},"aggregations":{"my-cardinality-aggregation-name":{"value":46}}}

### 4.4.3. 버킷 집계
* 버킷
  * 문서를 특정 기준으로 쪼개어 나눈 부분 집합
* 버킷 집계 시 버킷 생성 및 각 버킷에 포함된 문서 대상으로 하위 집계 (sub-aggregation) 수행 가능

#### range 집계
* 지정된 필드값 기준으로 문서를 원하는 버킷 구간으로 쪼갬
* `range` 밑에 `aggs` 하나 더 포함해 하위 집계 수행
* 버킷 하위 문서 수는 `doc_count`에서 확인 가능
* 하위 집계를 추가하지 않은 경우 `doc_count`만 집계
* 하위 집계에 버킷 집계를 넣을 수 있으나 하위 집계 깊이가 너무 깊어질 경우 성능에 문제가 발생할 수 있음

In [4]:
%%bash
curl -X GET --location "http://localhost:9200/kibana_sample_data_flights/_search" \
    -H "Content-Type: application/json" \
    -d "{
          \"size\": 0,
          \"query\": {
            \"match_all\": {}
          },
          \"aggs\": {
            \"distance-kilometers-range\": {
              \"range\": {
                \"field\": \"DistanceKilometers\",
                \"ranges\": [
                    {
                        \"to\": 5000
                    },
                    {
                        \"from\": 5000,
                        \"to\": 10000
                    },
                    {
                        \"from\": 10000
                    }
                ]
              },
              \"aggs\": {
                \"average-ticket-price\": {
                    \"avg\": {
                        \"field\": \"AvgTicketPrice\"
                    }
                }
              }
            }
          }
        }"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
curl: (7) Failed to connect to localhost port 9200 after 0 ms: Couldn't connect to server


CalledProcessError: Command 'b'curl -X GET --location "http://localhost:9200/kibana_sample_data_flights/_search" \\\n    -H "Content-Type: application/json" \\\n    -d "{\n          \\"size\\": 0,\n          \\"query\\": {\n            \\"match_all\\": {}\n          },\n          \\"aggs\\": {\n            \\"distance-kilometers-range\\": {\n              \\"range\\": {\n                \\"field\\": \\"DistanceKilometers\\",\n                \\"ranges\\": [\n                    {\n                        \\"to\\": 5000\n                    },\n                    {\n                        \\"from\\": 5000,\n                        \\"to\\": 10000\n                    },\n                    {\n                        \\"from\\": 10000\n                    }\n                ]\n              },\n              \\"aggs\\": {\n                \\"average-ticket-price\\": {\n                    \\"avg\\": {\n                        \\"field\\": \\"AvgTicketPrice\\"\n                    }\n 

#### date_range 집계
* 아래 내용 제외 `range` 집계와 유사 
  * `date` 타입 필드를 대상으로 사용
  * `from`과 `to`에 간단한 날짜 시간 계산식 사용 가능
* 각 샤드에 집계 요청이 분산되어 들어올 경우, 그 내용을 샤드 캐시에 올려, 동일한 요청이 같은 샤드로 들어오면 캐시를 활용
  * `now`가 포함된 요청은 캐시되지 않음
  * 새로운 데이터가 들어왔을 때에도 캐시 무효화됨

In [ ]:
%%bash
curl -X GET --location "http://localhost:9200/kibana_sample_data_ecommerce/_search" \
    -H "Content-Type: application/json" \
    -d "{
          \"size\": 0,
          \"query\": {
            \"match_all\": {}
          },
          \"aggs\": {
            \"date-range-aggs\": {
              \"date_range\": {
                \"field\": \"order_date\",
                \"ranges\": [
                    {
                        \"to\": \"now-10d/d\"
                    },
                    {
                        \"from\": \"now-10d/d\",
                        \"to\": \"now\"
                    },
                    {
                        \"from\": \"now\"
                    }
                ]
              }
            }
          }
        }"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1253    0   654  100   599  98926  90606 --:--:-- --:--:-- --:--:--  203k


{"took":3,"timed_out":false,"_shards":{"total":1,"successful":1,"skipped":0,"failed":0},"hits":{"total":{"value":4675,"relation":"eq"},"max_score":null,"hits":[]},"aggregations":{"date-range-aggs":{"buckets":[{"key":"*-2024-04-23T00:00:00.000Z","to":1.7138304E12,"to_as_string":"2024-04-23T00:00:00.000Z","doc_count":2822},{"key":"2024-04-23T00:00:00.000Z-2024-05-03T05:15:10.898Z","from":1.7138304E12,"from_as_string":"2024-04-23T00:00:00.000Z","to":1.714713310898E12,"to_as_string":"2024-05-03T05:15:10.898Z","doc_count":1591},{"key":"2024-05-03T05:15:10.898Z-*","from":1.714713310898E12,"from_as_string":"2024-05-03T05:15:10.898Z","doc_count":262}]}}}

#### histogram 집계
* 버킷 간격 (`interval`)을 지정해 경계를 나눔
* `offset`을 별도로 지정하지 않을 경우 0을 기준으로 히스토그램 계급을 나눔
* `min_doc_count` 지정 시 버킷 내 문서 개수가 일정 이하인 버킷 결과에서 제외 가능

In [ ]:
%%bash
curl -X GET --location "http://localhost:9200/kibana_sample_data_flights/_search" \
    -H "Content-Type: application/json" \
    -d "{
          \"size\": 0,
          \"query\": {
            \"match_all\": {}
          },
          \"aggs\": {
            \"my-histogram\": {
              \"histogram\": {
                \"field\": \"DistanceKilometers\",
                \"interval\": 1000,
                \"offset\": 50 
              }
            }
          }
        }"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1200    0   874  100   326  45990  17154 --:--:-- --:--:-- --:--:-- 66666


{"took":14,"timed_out":false,"_shards":{"total":1,"successful":1,"skipped":0,"failed":0},"hits":{"total":{"value":10000,"relation":"gte"},"max_score":null,"hits":[]},"aggregations":{"my-histogram":{"buckets":[{"key":-950.0,"doc_count":641},{"key":50.0,"doc_count":1226},{"key":1050.0,"doc_count":1106},{"key":2050.0,"doc_count":521},{"key":3050.0,"doc_count":232},{"key":4050.0,"doc_count":329},{"key":5050.0,"doc_count":428},{"key":6050.0,"doc_count":1097},{"key":7050.0,"doc_count":1729},{"key":8050.0,"doc_count":1361},{"key":9050.0,"doc_count":1414},{"key":10050.0,"doc_count":644},{"key":11050.0,"doc_count":648},{"key":12050.0,"doc_count":325},{"key":13050.0,"doc_count":290},{"key":14050.0,"doc_count":280},{"key":15050.0,"doc_count":296},{"key":16050.0,"doc_count":312},{"key":17050.0,"doc_count":34},{"key":18050.0,"doc_count":72},{"key":19050.0,"doc_count":29}]}}}

#### date_histogram 집계
* `histogram` 집계와 유사하지만 `date` 타입 필드 사용
* `interval` 대신 `calendar_interval`이나 `fixed_interval` 사용
  * `calendar_interval`
    * 분 단위: `minute`, `1m`
    * 일 단위: `day`, `1d`
    * 월 단위: `month`, `1M`
    * 분기 단위: `quarter`, `1q`
    * 연 단위: `year`, `1y`
  * `fixed_interval`
    * 1개 단위가 아닐 때 사용
    * `ms`, `s`, `m`, `h`, `d` 단위 사용 가능      
* `offset`과 `min_doc_count` 설정 가능 

In [ ]:
%%bash
curl -X GET --location "http://localhost:9200/kibana_sample_data_ecommerce/_search" \
    -H "Content-Type: application/json" \
    -d "{
          \"size\": 0,
          \"query\": {
            \"match_all\": {}
          },
          \"aggs\": {
            \"my-date-histogram\": {
              \"date_histogram\": {
                \"field\": \"order_date\",
                \"calendar_interval\": \"day\"
              }
            }
          }
        }"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3033    0  2726  100   307  44988   5066 --:--:-- --:--:-- --:--:-- 50550


{"took":55,"timed_out":false,"_shards":{"total":1,"successful":1,"skipped":0,"failed":0},"hits":{"total":{"value":4675,"relation":"eq"},"max_score":null,"hits":[]},"aggregations":{"my-date-histogram":{"buckets":[{"key_as_string":"2024-04-04T00:00:00.000Z","key":1712188800000,"doc_count":146},{"key_as_string":"2024-04-05T00:00:00.000Z","key":1712275200000,"doc_count":153},{"key_as_string":"2024-04-06T00:00:00.000Z","key":1712361600000,"doc_count":143},{"key_as_string":"2024-04-07T00:00:00.000Z","key":1712448000000,"doc_count":140},{"key_as_string":"2024-04-08T00:00:00.000Z","key":1712534400000,"doc_count":139},{"key_as_string":"2024-04-09T00:00:00.000Z","key":1712620800000,"doc_count":157},{"key_as_string":"2024-04-10T00:00:00.000Z","key":1712707200000,"doc_count":145},{"key_as_string":"2024-04-11T00:00:00.000Z","key":1712793600000,"doc_count":152},{"key_as_string":"2024-04-12T00:00:00.000Z","key":1712880000000,"doc_count":163},{"key_as_string":"2024-04-13T00:00:00.000Z","key":171296640

#### terms 집계
* 지정한 필드에 대해 가장 빈도수 높은 term 순서대로 버킷 생성
* `size` 통해 최대 생성 버킷 수 결정
* 각 샤드에서 `size` 수 만큼 term 빈도수 집계해 합산한 후 `size` 개수만큼 버킷을 뽑음
  * `size` 개수와 문서의 분포에 따라 `doc_count` 및 하위 집계 결과가 정확하지 않을 수 있음
  * 고유한 `term` 수가 `size`보다 크다면, 상위에 뽑힐 `term`이 최종 결과에 포함되지 않을 수 있음
  * `size` 상향 조정 시 정확도  
* `doc_count_error_upper_bound` 오차 상한선을 뜻함
* `sum_other_doc_count` 최종적으로 버킷에 포함되지 않은 문서 수를 뜻함 
* 모든 `term`에 대하여 집계가 필요한 경우 `composite` 집계 사용 추천

In [ ]:
%%bash
curl -X GET --location "http://localhost:9200/kibana_sample_data_logs/_search" \
    -H "Content-Type: application/json" \
    -d "{
          \"size\": 0,
          \"query\": {
            \"match_all\": {}
          },
          \"aggs\": {
            \"my-term-aggs\": {
              \"terms\": {
                \"field\": \"host.keyword\",
                \"size\": 10
              }
            }
          }
        }"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   750    0   471  100   279  23364  13839 --:--:-- --:--:-- --:--:-- 37500


{"took":16,"timed_out":false,"_shards":{"total":1,"successful":1,"skipped":0,"failed":0},"hits":{"total":{"value":10000,"relation":"gte"},"max_score":null,"hits":[]},"aggregations":{"my-term-aggs":{"doc_count_error_upper_bound":0,"sum_other_doc_count":0,"buckets":[{"key":"artifacts.elastic.co","doc_count":6488},{"key":"www.elastic.co","doc_count":4779},{"key":"cdn.elastic-elastic-elastic.org","doc_count":2255},{"key":"elastic-elastic-elastic.org","doc_count":552}]}}}

#### composite 집계
* `sources`로 지정된 하위 집계 버킷을 전부를 페이지네이션을 이용해 효율적으로 순회하는 집계
  * `terms`, `histogram`, `date_histogram` 등 일부 집계만 가능 
* `sources`에 하위 집계 여러 개 지정한 후 조합된 버킷 생성 가능
* `size` 페이지네이션 한 번에 몇 개의 버킷을 반환할지 지정